In [6]:
#import library
import xarray as xr
import netCDF4
import cartopy
import cftime
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pyproj
import numpy as np

In [30]:

def extractWsFromModel(fileinput, wslat,wslon):
    ds = xr.open_dataset(fileinput, decode_times=False)
    # Extract the time variable
    time_var = ds["time"]

    # Get the time units, and remove the non-standard prefix
    units = time_var.attrs['units'].replace('fire ignition: ', '')

    # Convert the time variable using cftime
    times = cftime.num2date(time_var.values, units=units, calendar='standard')

    # Convert to pandas datetime if needed
    times_as_datetime = [datetime(year=t.year, month=t.month, day=t.day, 
                              hour=t.hour, minute=t.minute, second=t.second,microsecond=t.microsecond)
                     for t in times]


    # Replace the time variable in the dataset with the converted times
    ds["time"] = ("time", times_as_datetime)

    lat = ds['lat'].values[:,0]
    lon = ds['lon'].values[0,:]

    # Calculate the absolute differences between the werather stations coordinates and the grid points
    idx_lat = np.abs(lat - wslat).argmin()
    idx_lon = np.abs(lon - wslon).argmin()

    df = ds.isel(x=idx_lon,y=idx_lat)[['um10','vm10','t2m','gust10']].to_dataframe()

    df['wdir'] = (np.arctan2(df['um10'], df['vm10']) * 180 / np.pi + 360) % 360

    df['wspeed'] = np.sqrt(df['um10']**2 + df['vm10']**2)

    return df.drop(columns=['x','y'])


In [31]:

fileinput = '/data/IMFSE/PythonCourse/MNH/FCAST_model3.nc'

# Coordinates of the weather stations
wslat = 41.83094
wslon = 1.84699

extractWsFromModel(fileinput, wslat,wslon)

,um10,vm10,t2m,gust10,wdir,wspeed
time,,,,,,
2022-07-16 00:00:00,0.284463,-0.454374,1.000000e+20,0.936074,147.951146,0.536074
2022-07-16 01:00:00,-0.657117,-0.278774,1.000000e+20,1.113805,247.011516,0.713805
2022-07-16 02:00:00,-0.252196,-0.282423,1.000000e+20,0.778636,221.763988,0.378636
2022-07-16 03:00:00,-0.507731,-0.888757,1.000000e+20,1.423562,209.738534,1.023562
2022-07-16 04:00:00,-0.693409,-0.248774,1.000000e+20,1.192280,250.263559,0.736684
2022-07-16 05:00:00,0.270433,-0.094160,1.000000e+20,0.717645,109.197153,0.286357
2022-07-16 06:00:00,-0.117394,0.086022,1.000000e+20,0.742919,306.232475,0.145538
2022-07-16 07:00:00,-0.454005,0.059391,1.000000e+20,1.747353,277.452910,0.457873
2022-07-16 08:00:00,-0.597392,-0.165096,1.000000e+20,2.437446,254.551313,0.619785


In [32]:
ds

<xarray.Dataset> Size: 7GB
Dimensions:       (x: 150, y: 150, z: 50, time: 49)
Coordinates:
  * x             (x) float32 600B 5.344e+04 5.352e+04 ... 6.528e+04 6.536e+04
  * y             (y) float32 600B 7.344e+04 7.352e+04 ... 8.528e+04 8.536e+04
  * z             (z) float32 200B 15.0 46.5 81.15 ... 1.736e+04 1.826e+04
  * time          (time) datetime64[ns] 392B 2022-07-16 ... 2022-07-18
Data variables: (12/31)
    lat           (y, x) float64 180kB ...
    lon           (y, x) float64 180kB ...
    dx            (z, y, x) float64 9MB ...
    dy            (z, y, x) float64 9MB ...
    dz            (z, y, x) float64 9MB ...
    topography    (y, x) float64 180kB ...
    ...            ...
    vm10          (time, y, x) float64 9MB ...
    gust10        (time, y, x) float64 9MB ...
    lambda2       (time, z, y, x) float64 441MB ...
    acprecip      (time, z, y, x) float64 441MB ...
    cloudfrac     (time, z, y, x) float64 441MB ...
    t2m           (time, y, x) float64 9MB ...
Attributes:
    description:  MNH Output for simulation FCAST and model3
    created:      R. Paugam
    company:      DoWeNeedOne
    title:        MNH Output